# Q1

A stochastic process is a collection of random variables indexed over time or space, describing systems that evolve randomly.A Gaussian Process (GP) is a specific type of stochastic process where any finite collection of variables follows a multivariate Gaussian distribution.GPs are useful for non-parametric regression and probabilistic modeling in machine learning.Variational Inference (VI) is a technique for approximating probability distributions, often used in Bayesian machine learning.ELBO (Evidence Lower Bound) is an objective function in VI that helps approximate an intractable posterior distribution. Instead of directly computing the posterior, VI optimizes ELBO to find a simpler approximation.


# Q2

In [ ]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd

# Load real-world temperature dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
data = pd.read_csv(url, parse_dates=["Date"], index_col="Date")
data = data.resample("M").mean()  # Convert to monthly averages
X = np.arange(len(data)).reshape(-1, 1)  # Time indices as feature
y = data["Temp"].values  # Temperature values

# Standardize the target variable for numerical stability
y_mean, y_std = y.mean(), y.std()
y = (y - y_mean) / y_std  # Normalize temperature values

# Define a Gaussian Process Model in PyMC
with pm.Model() as gp_model:
    # Define GP components
    mean_func = pm.gp.mean.Zero()  # Zero mean function
    cov_func = pm.gp.cov.ExpQuad(input_dim=1, ls=10)  # Squared exponential kernel
    
    # Define GP with keyword arguments
    gp = pm.gp.Marginal(mean_func=mean_func, cov_func=cov_func)

    # Likelihood (Noise term)
    sigma = pm.HalfNormal("sigma", sigma=1)
    
    # Define the Marginal Likelihood
    y_obs = gp.marginal_likelihood("y_obs", X=X, y=y, noise=sigma)

    # Perform inference using MCMC
    trace = pm.sample(1000, return_inferencedata=True, chains=2, target_accept=0.9)

# Plot the results
az.plot_trace(trace)
plt.show()


# Q3

In [ ]:
import pymc as pm
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load dataset (MNIST)
(X_train, y_train), (_, _) = mnist.load_data()
X_train = X_train.reshape(-1, 28 * 28) / 255.0  # Flatten images & normalize
y_train = y_train.astype("int")

# Select a smaller dataset for fast training
X_train, y_train = X_train[:1000], y_train[:1000]

# Define a Bayesian Neural Network in PyMC
with pm.Model() as bnn:
    # Priors for the first layer
    W1 = pm.Normal("W1", 0, 1, shape=(28 * 28, 128))
    b1 = pm.Normal("b1", 0, 1, shape=(128,))
    
    # Hidden layer
    hidden = pm.math.tanh(pm.math.dot(X_train, W1) + b1)
    
    # Priors for the second layer
    W2 = pm.Normal("W2", 0, 1, shape=(128, 10))
    b2 = pm.Normal("b2", 0, 1, shape=(10,))
    
    # Output layer (Softmax for classification)
    logits = pm.math.dot(hidden, W2) + b2
    y_obs = pm.Categorical("y_obs", pm.math.softmax(logits), observed=y_train)
    
    # Variational Inference (ADVI)
    approx = pm.fit(n=5000, method="advi")  # Run ADVI optimization
    trace = approx.sample(1000)  # Sample from the approximate posterior

# Plot the ELBO loss curve
plt.plot(approx.hist)
plt.xlabel("Iteration")
plt.ylabel("ELBO Loss")
plt.title("Variational Inference Training")
plt.show()
